In [26]:
import pandas as pd
import numpy as np
import random
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [32]:
pwd

'/home/jm/Desktop/lambda_projects/pitch_predict/notebooks'

In [33]:
ls

Fill_Pitch_Type_Nans_&_Make_Pitch_Category_Feature.ipynb
make_count_features.ipynb
pitch_data_download.ipynb
Untitled.ipynb


In [27]:
fname = 'pitches_2017.pkl'
df_17 = pd.read_pickle(fname, compression='zip')

#convert the pitch type for UN (unknown) to np.nan
df_17['pitch_type'] = df_17['pitch_type'].replace({'UN':np.nan})

#fix some faulty data that has number of balls listed as 4:
df_17['balls'] = df_17['balls'].replace({4.0: 3.0})

In [28]:
df_17.head()

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,fielder_2,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,593,FC,2017-10-01,91.9,-1.1948,5.9921,Roberto Osuna,595885.0,532077.0,field_out,hit_into_play,1.0,Greg Bird pops out to shortstop Richard Urena.,L,R,NYY,TOR,X,6.0,popup,3.0,2.0,2017.0,-0.0631,1.5401,-0.4785,3.2490,NaN,NaN,NaN,2.0,9.0,Bot,108.43,103.14,607257.0,NaN,1.9309,-133.4022,-4.2967,-1.1718,29.8138,-13.3776,3.2855,1.4775,236.0,81.3,51.0,90.975,2422.0,5.941,492528.0,532077.0,607257.0,475253.0,572365.0,446381.0,620446.0,485567.0,606192.0,608701.0,54.5564,0.023,0.023,0.0,1.0,0.0,0.0,3.0,63.0,6.0,Cutter,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,Infield shift,Strategic
1,614,FC,2017-10-01,91.8,-1.0777,6.1772,Roberto Osuna,595885.0,532077.0,NaN,foul,3.0,NaN,L,R,NYY,TOR,S,NaN,NaN,3.0,1.0,2017.0,0.3124,1.1742,0.5540,2.8765,NaN,NaN,NaN,2.0,9.0,Bot,NaN,NaN,607257.0,NaN,3.4438,-133.4490,-4.9985,3.0138,26.8527,-17.4553,3.2855,1.4775,NaN,NaN,NaN,91.493,2415.0,6.005,492528.0,532077.0,607257.0,475253.0,572365.0,446381.0,620446.0,485567.0,606192.0,608701.0,54.4926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,5.0,Cutter,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,Infield shift,Strategic
2,618,FF,2017-10-01,94.7,-0.6208,6.3387,Roberto Osuna,595885.0,532077.0,NaN,ball,12.0,NaN,L,R,NYY,TOR,B,NaN,NaN,2.0,1.0,2017.0,-0.2663,1.8419,1.5362,2.7561,NaN,NaN,NaN,2.0,9.0,Bot,NaN,NaN,607257.0,NaN,6.1756,-137.3397,-7.6583,-4.7051,30.2109,-7.5693,3.4375,1.6782,20.0,75.0,-8.0,94.150,2450.0,6.126,492528.0,532077.0,607257.0,475253.0,572365.0,446381.0,620446.0,485567.0,606192.0,608701.0,54.3712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,4.0,4-Seam Fastball,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,Infield shift,Strategic
3,636,CH,2017-10-01,82.8,-0.9901,6.0626,Roberto Osuna,595885.0,532077.0,NaN,ball,13.0,NaN,L,R,NYY,TOR,B,NaN,NaN,1.0,1.0,2017.0,-1.2683,1.3751,-0.2105,0.8006,NaN,NaN,NaN,2.0,9.0,Bot,NaN,NaN,607257.0,NaN,4.2193,-120.0613,-8.1302,-13.0569,22.8897,-17.5538,3.4515,1.7117,NaN,NaN,NaN,82.564,2063.0,6.284,492528.0,532077.0,607257.0,475253.0,572365.0,446381.0,620446.0,485567.0,606192.0,608701.0,54.2138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,3.0,Changeup,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,Infield shift,Strategic
4,650,FT,2017-10-01,95.9,-0.8903,6.1341,Roberto Osuna,595885.0,532077.0,NaN,ball,13.0,NaN,L,R,NYY,TOR,B,NaN,NaN,0.0,1.0,2017.0,-1.1935,1.4982,-0.7871,0.8732,NaN,NaN,NaN,2.0,9.0,Bot,NaN,NaN,607257.0,NaN,2.9467,-139.0987,-11.6276,-16.1537,30.5031,-10.3668,3.6075,1.7702,NaN,NaN,NaN,95.761,2554.0,6.330,492528.0,532077.0,607257.0,475253.0,572365.0,446381.0,620446.0,485567.0,606192.0,608701.0,54.1680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,2.0,2-Seam Fastball,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,Infield shift,Strategic


In [29]:
def make_count_features(df):
    df = df.copy()
    
    #convert balls and strikes columns to strings (first to int)
    df['balls'] = df['balls'].astype('int').astype('str')
    df['strikes'] = df['strikes'].astype('int').astype('str')

    #create count feature:
    df['_count'] = df['balls'] + df['strikes']

    #create count category feature:
    count_map = {'00':'neutral', '21':'neutral', '32':'neutral', '10':'behind', '20':'behind',
             '30':'behind', '31':'behind', '01':'ahead', '02':'ahead', '11':'ahead',
             '12':'ahead', '22':'ahead'}
    df['count_category'] = df['_count'].replace(count_map)
    df = df.drop(columns=['balls', 'strikes'])
    return df



In [30]:
df_17 = make_count_features(df_17)

In [31]:
df_17.head()

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,stand,p_throws,home_team,away_team,type,hit_location,bb_type,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,fielder_2,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,_count,count_category
0,593,FC,2017-10-01,91.9,-1.1948,5.9921,Roberto Osuna,595885.0,532077.0,field_out,hit_into_play,1.0,Greg Bird pops out to shortstop Richard Urena.,L,R,NYY,TOR,X,6.0,popup,2017.0,-0.0631,1.5401,-0.4785,3.2490,NaN,NaN,NaN,2.0,9.0,Bot,108.43,103.14,607257.0,NaN,1.9309,-133.4022,-4.2967,-1.1718,29.8138,-13.3776,3.2855,1.4775,236.0,81.3,51.0,90.975,2422.0,5.941,492528.0,532077.0,607257.0,475253.0,572365.0,446381.0,620446.0,485567.0,606192.0,608701.0,54.5564,0.023,0.023,0.0,1.0,0.0,0.0,3.0,63.0,6.0,Cutter,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,Infield shift,Strategic,32,neutral
1,614,FC,2017-10-01,91.8,-1.0777,6.1772,Roberto Osuna,595885.0,532077.0,NaN,foul,3.0,NaN,L,R,NYY,TOR,S,NaN,NaN,2017.0,0.3124,1.1742,0.5540,2.8765,NaN,NaN,NaN,2.0,9.0,Bot,NaN,NaN,607257.0,NaN,3.4438,-133.4490,-4.9985,3.0138,26.8527,-17.4553,3.2855,1.4775,NaN,NaN,NaN,91.493,2415.0,6.005,492528.0,532077.0,607257.0,475253.0,572365.0,446381.0,620446.0,485567.0,606192.0,608701.0,54.4926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,5.0,Cutter,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,Infield shift,Strategic,31,behind
2,618,FF,2017-10-01,94.7,-0.6208,6.3387,Roberto Osuna,595885.0,532077.0,NaN,ball,12.0,NaN,L,R,NYY,TOR,B,NaN,NaN,2017.0,-0.2663,1.8419,1.5362,2.7561,NaN,NaN,NaN,2.0,9.0,Bot,NaN,NaN,607257.0,NaN,6.1756,-137.3397,-7.6583,-4.7051,30.2109,-7.5693,3.4375,1.6782,20.0,75.0,-8.0,94.150,2450.0,6.126,492528.0,532077.0,607257.0,475253.0,572365.0,446381.0,620446.0,485567.0,606192.0,608701.0,54.3712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,4.0,4-Seam Fastball,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,Infield shift,Strategic,21,neutral
3,636,CH,2017-10-01,82.8,-0.9901,6.0626,Roberto Osuna,595885.0,532077.0,NaN,ball,13.0,NaN,L,R,NYY,TOR,B,NaN,NaN,2017.0,-1.2683,1.3751,-0.2105,0.8006,NaN,NaN,NaN,2.0,9.0,Bot,NaN,NaN,607257.0,NaN,4.2193,-120.0613,-8.1302,-13.0569,22.8897,-17.5538,3.4515,1.7117,NaN,NaN,NaN,82.564,2063.0,6.284,492528.0,532077.0,607257.0,475253.0,572365.0,446381.0,620446.0,485567.0,606192.0,608701.0,54.2138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,3.0,Changeup,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,Infield shift,Strategic,11,ahead
4,650,FT,2017-10-01,95.9,-0.8903,6.1341,Roberto Osuna,595885.0,532077.0,NaN,ball,13.0,NaN,L,R,NYY,TOR,B,NaN,NaN,2017.0,-1.1935,1.4982,-0.7871,0.8732,NaN,NaN,NaN,2.0,9.0,Bot,NaN,NaN,607257.0,NaN,2.9467,-139.0987,-11.6276,-16.1537,30.5031,-10.3668,3.6075,1.7702,NaN,NaN,NaN,95.761,2554.0,6.330,492528.0,532077.0,607257.0,475253.0,572365.0,446381.0,620446.0,485567.0,606192.0,608701.0,54.1680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,2.0,2-Seam Fastball,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,Infield shift,Strategic,01,ahead
